In [1]:
import pandas as pd
import os
import chardet
import numpy as np
import math

In [2]:
# change folder for your data directory
import os

folder = 'C:\\Users\\kirill.n.cherkasov\\Desktop\\Pyprojects\\skillfactory\\data'
log_file = os.path.join(folder, 'log.xls')
users_file = os.path.join(folder, 'users.xls')

# Определяем кодировки

with open(log_file, 'rb') as f:
    logs_encoding = chardet.detect(f.read())['encoding']
    print(f'Logs file encoding is {logs_encoding}')
with open(users_file, 'rb') as f:
    users_encoding = chardet.detect(f.read())['encoding']
    print(f'Users file encoding is {users_encoding}')

Logs file encoding is utf-8
Users file encoding is KOI8-R


In [3]:
df_logs = pd.read_csv(log_file, header=None)
df_users = pd.read_csv(users_file, encoding=users_encoding)

In [4]:
# Переименовываем колонки

df_logs.columns = ('user_id', 'time', 'bet', 'win')
df_logs.head()

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


In [5]:
# Количество пустых значений в столбце

df_logs['time'].isna().value_counts()

False    985
True      15
Name: time, dtype: int64

### В следующих строках df_logs не меняется, создаётся его копия

In [6]:
# Дропаем все столбцы, где есть хоть одно пустое значение

df_logs_bak = df_logs.copy()
df_logs_bak.dropna(axis=1, inplace=True)
df_logs_bak.head()

,user_id
0,Запись пользователя № - user_919
1,Запись пользователя № - user_973
2,Запись пользователя № - user_903
3,Запись пользователя № - user_954
4,Запись пользователя № - user_954


In [7]:
# Дропаем все строки, где есть хоть одно пустое значение

df_logs_bak2 = df_logs.copy()
df_logs_bak2.dropna(axis=0, inplace=True)
df_logs_bak2.count()

user_id    133
time       133
bet        133
win        133
dtype: int64

In [8]:
# Дропаем целиком только определённые столбцы, которые имеют пропущенные значения

df_logs_bak3 = df_logs.copy()
for col in ['time', 'user_id']:
    try:
        if df_logs_bak3[col].isna().value_counts()[True] >= 0:
            df_logs_bak3.drop(col, axis=1, inplace=True)
    except KeyError:
        pass
df_logs_bak3.head()

,user_id,bet,win
0,Запись пользователя № - user_919,NaN,NaN
1,Запись пользователя № - user_973,NaN,NaN
2,Запись пользователя № - user_903,NaN,NaN
3,Запись пользователя № - user_954,NaN,NaN
4,Запись пользователя № - user_954,NaN,NaN


In [9]:
df_logs.drop_duplicates(subset=['user_id', 'time']).count()

user_id    986
time       985
bet        472
win        133
dtype: int64

In [10]:
example = '[2019-01-01 14:06:51'
def parse_time(s: str):
    if str(s).startswith('['):
        return s[1:]
parse_time(example)

'2019-01-01 14:06:51'

In [11]:
df_logs.time = df_logs.time.apply(parse_time)

In [12]:
df_logs.head()

,user_id,time,bet,win
0,Запись пользователя № - user_919,2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN


In [13]:
time_series = df_logs.time.dropna()

In [14]:
time_series.isna().value_counts()

False    985
Name: time, dtype: int64

In [15]:
time_series = pd.to_datetime(time_series)

In [16]:
time_series.max()

Timestamp('2019-04-20 18:10:07')

In [17]:
time_series.head(5)

0   2019-01-01 14:06:51
1   2019-01-01 14:51:16
2   2019-01-01 16:31:16
3   2019-01-01 17:17:51
4   2019-01-01 21:31:18
Name: time, dtype: datetime64[ns]

In [18]:
log = pd.read_csv(os.path.join(folder, "log.xls"))  
log = log.dropna()  
log.columns = ['user_id', 'time', 'bet', 'win']  
log['date_time'] = pd.to_datetime(log['time'].apply(lambda x: x[1:]))
log['minutes'] = log['date_time'].apply(lambda x: x.minute)
log['minutes'].head() 

13     57
28     59
150    54
188    34
204    26
Name: minutes, dtype: int64

In [19]:
log['minutes'].value_counts().head()

36    7
57    5
44    5
18    5
35    4
Name: minutes, dtype: int64

In [21]:
log = pd.read_csv(os.path.join(folder, "log.xls"))  
log = log.dropna()  
log.columns = ['user_id', 'time', 'bet', 'win']  
log['date_time'] = pd.to_datetime(log['time'].apply(lambda x: x[1:]))
log['month'] = log['date_time'].apply(lambda x: x.month)
log['month'].value_counts()

3    57
4    51
2    16
1     9
Name: month, dtype: int64

In [22]:
log = pd.read_csv(os.path.join(folder, "log.xls"))
log.columns = ['user_id', 'time', 'bet', 'win'] 
ts = log['time'].dropna()  
dts = pd.to_datetime(ts.apply(lambda x: x[1:]))
mts = dts.apply(lambda x: x.month)
mts.value_counts()

1    291
3    264
2    259
4    170
Name: time, dtype: int64

In [23]:
df_logs['datetime'] = pd.to_datetime(df_logs['time'])

In [24]:
df_logs.head()

,user_id,time,bet,win,datetime
0,Запись пользователя № - user_919,2019-01-01 14:06:51,NaN,NaN,2019-01-01 14:06:51
1,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN,2019-01-01 14:51:16
2,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN,2019-01-01 16:31:16
3,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN,2019-01-01 17:17:51
4,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN,2019-01-01 21:31:18


In [25]:
df_logs['weekday'] = df_logs['datetime'].dt.weekday

In [26]:
df_logs.head()

,user_id,time,bet,win,datetime,weekday
0,Запись пользователя № - user_919,2019-01-01 14:06:51,NaN,NaN,2019-01-01 14:06:51,1.0
1,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN,2019-01-01 14:51:16,1.0
2,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN,2019-01-01 16:31:16,1.0
3,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN,2019-01-01 17:17:51,1.0
4,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN,2019-01-01 21:31:18,1.0


In [27]:
df_logs['weekday'].value_counts()

5.0    152
2.0    150
1.0    150
0.0    135
4.0    135
3.0    132
6.0    131
Name: weekday, dtype: int64

In [28]:
df_logs['weekday'].isna().value_counts()

False    985
True      15
Name: weekday, dtype: int64

In [29]:
def daytime(dt):
    if pd.isna(dt):
        return np.nan
    h = dt.hour
    if 0 <= h <= 5:
        return 'ночь'
    if 6 <= h <= 11:
        return 'утро'
    if 12 <= h <= 17:
        return 'день'
    if 18 <= h <= 23:
        return 'вечер'

In [30]:
df_logs['daytime'] = df_logs['datetime'].apply(daytime)

In [31]:
df_logs.head()

,user_id,time,bet,win,datetime,weekday,daytime
0,Запись пользователя № - user_919,2019-01-01 14:06:51,NaN,NaN,2019-01-01 14:06:51,1.0,день
1,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN,2019-01-01 14:51:16,1.0,день
2,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN,2019-01-01 16:31:16,1.0,день
3,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN,2019-01-01 17:17:51,1.0,день
4,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN,2019-01-01 21:31:18,1.0,вечер


In [32]:
df_logs.daytime.value_counts()

ночь     265
утро     253
день     240
вечер    227
Name: daytime, dtype: int64

In [33]:
log = pd.read_csv(os.path.join(folder, 'log.xls'), header=None)
log.columns = ('user_id', 'time', 'bet', 'win')
log.dropna(inplace=True, subset=['time'])
log['time'] = pd.to_datetime(log['time'].apply(lambda x: x[1:]))
log['hour'] = log['time'].apply(lambda x: x.hour)

In [34]:
log.count()

user_id    985
time       985
bet        472
win        133
hour       985
dtype: int64

In [35]:
df_logs['bet'].fillna(0, inplace=True)

In [36]:
df_logs.bet.value_counts()[0]

515

In [37]:
def fillna_win(row):
    if not pd.isna(row.win):
        return row
    if row.bet == 0:
        row.win = 0
        return row
    row.win = -row.bet
    return row

In [38]:
df_logs = df_logs.apply(lambda row: fillna_win(row), axis=1)

In [39]:
df_logs.head()

,user_id,time,bet,win,datetime,weekday,daytime
0,Запись пользователя № - user_919,2019-01-01 14:06:51,0.0,0.0,2019-01-01 14:06:51,1.0,день
1,Запись пользователя № - user_973,2019-01-01 14:51:16,0.0,0.0,2019-01-01 14:51:16,1.0,день
2,Запись пользователя № - user_903,2019-01-01 16:31:16,0.0,0.0,2019-01-01 16:31:16,1.0,день
3,Запись пользователя № - user_954,2019-01-01 17:17:51,0.0,0.0,2019-01-01 17:17:51,1.0,день
4,Запись пользователя № - user_954,2019-01-01 21:31:18,0.0,0.0,2019-01-01 21:31:18,1.0,вечер


In [40]:
df_logs[df_logs.win < 0].win.count()

347

In [41]:
df_logs['net'] = df_logs['win'] - df_logs['bet']

Задание 1
1/1 point (graded)
Используйте модифицированный в прошлой секции датасет log.csv. Результат запишите числом в поле ниже.

Подсказка - можно использовать sum().

Создайте признак net, хранящий сумму выигрыша с учетом ставки. Для этого из признака win поэлементно вычтите признак bet и запишите в новый столбец. После этого посчитайте, у скольких людей выигрыш положительный.

In [42]:
df_logs.net[df_logs.net > 0].count()

138

Задание 2
1/1 point (graded)
Используйте датасет log.csv, получившийся в результате выполнения предыдущего задания. Посчитайте среднее значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

Подсказка: можно использовать mean().

In [43]:
df_logs.net[df_logs.net > 0].mean()

80253.33333333333

Задание 3
1 point possible (graded)
Используйте датасет log.csv, получившийся в результате выполнения первого задания этого блока. Посчитайте медианное значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

Подсказка: можно использовать median().

In [44]:
df_logs.net[df_logs.net > 0].median()

5347.0

In [45]:
log.bet.sum() / log.bet.dropna().shape[0]

6806.578389830508

In [46]:
log.bet.mean()

6806.578389830508

In [47]:
np.mean(log.bet)

6806.578389830508

In [48]:
log['bet'].dropna().mean()

6806.578389830508

#### Задание 1

Используйте модифицированный исходный датасет log.csv.

При модификации датасета log.csv, пропущенные значения в столбцах bet и win замените на 0, cоздайте столбец net, хранящий сумму выигрыша с учетом ставки (для этого из столбца win поэлементно вычтите столбец bet и запишите в новый столбец).

Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. Для этого поделите количество ставок (значений больше 0) на общее количество посещений конторы. Результат округлите до одного знака после запятой.

In [49]:
df_logs.win[df_logs.win < 0] = 0

C:\Users\kirill.n.cherkasov\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
df_logs.net = df_logs.win - df_logs.bet

In [51]:
df_logs.bet[df_logs.bet > 0].count() / df_logs.bet.count() * 100

48.5

#### Задание 2
1 point possible (graded)
Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте среднее значение ставки (из столбца bet) в тех случаях, когда ставка была сделана. Результат округлите до целого, отбросив дробную часть.
Подсказка: можно использовать mean().

In [52]:
math.trunc(df_logs.bet[df_logs.bet > 0].mean())

6785

#### Задание 3
1 point possible (graded)
Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте средний выигрыш (из столбца net) в тех случаях, когда ставка была сделана. Результат округлите до целого, отбросив дробную часть.
Пояснение: выигрыш в данном случае означает изменение количества денег и может быть отрицательным. В таком случае это проигрыш.

Подсказка: можно использовать mean().

In [53]:
math.trunc(df_logs[df_logs.bet > 0].net.mean())

20421

#### Задание 4
1/1 point (graded)
Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте среднюю сумму потерь при проигрыше (из столбца net). Результат округлите до целого, отбросив дробную часть.
Пояснение: ответ должен быть дан в виде отрицательного числа.

Подсказка: можно использовать mean().

In [54]:
math.trunc(df_logs.net[df_logs.net < 0].mean())

-3372

#### Задание 5
1 point possible (graded)
Посчитайте, какой процент ставок заканчивается выигрышем, а какой - проигрышем. Сравните эти значения и ответьте, какое из них больше.

Выберите пункт из списка ниже.

Количество выигрышей от общего количества ставок

In [55]:
df_logs.net[df_logs.net > 0].count() / df_logs.net.count()

0.138

Количество проигрышей от общего количества ставок

In [56]:
df_logs.net[df_logs.net < 0].count() / df_logs.net.count()

0.347

Задание 6*
1 point possible (graded)
Давайте повторим то, что мы прошли в этой секции. Напишите код, который узнает, чему была равна минимальная ставка и сколько людей сделали такую ставку. Для этого:

1. загрузите датасет log.csv;

2. посчитайте, чему равна минимальная ставка;

3. посчитайте, сколько раз была сделана минимальная ставка, и запишите результат в переменную min_bet_amount в виде целого числа.

In [57]:
log = pd.read_csv(os.path.join(folder, 'log.xls'), header=None)
log.columns = ['user_id', 'time', 'bet', 'win']
log.bet[log.bet == log.bet.min()].count()

48

Разделяем userinfo на 3 независимых поля

In [59]:
df_users.head()

,Юзверь мейл Гео
0,User_943\tAccumanst@gmail.com\tИжевск
1,User_908\tAdvismowr@mail.ru\tИжевск
2,User_962\tAnachso@ukr.net\tКраснодар
3,User_973\tAntecia@inbox.ru\tПермь
4,User_902\tBalliaryva@ukr.net\t


In [60]:
df_logs = df_logs[df_logs.user_id != '#error']

In [61]:
df_logs.user_id = df_logs.user_id.str.split('-').apply(lambda x: x[1])

In [64]:
df_users.head()

,Юзверь мейл Гео
0,User_943\tAccumanst@gmail.com\tИжевск
1,User_908\tAdvismowr@mail.ru\tИжевск
2,User_962\tAnachso@ukr.net\tКраснодар
3,User_973\tAntecia@inbox.ru\tПермь
4,User_902\tBalliaryva@ukr.net\t


In [65]:
df_logs.head()

,user_id,time,bet,win,datetime,weekday,daytime,net
0,user_919,2019-01-01 14:06:51,0.0,0.0,2019-01-01 14:06:51,1.0,день,0.0
1,user_973,2019-01-01 14:51:16,0.0,0.0,2019-01-01 14:51:16,1.0,день,0.0
2,user_903,2019-01-01 16:31:16,0.0,0.0,2019-01-01 16:31:16,1.0,день,0.0
3,user_954,2019-01-01 17:17:51,0.0,0.0,2019-01-01 17:17:51,1.0,день,0.0
4,user_954,2019-01-01 21:31:18,0.0,0.0,2019-01-01 21:31:18,1.0,вечер,0.0


In [69]:
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 0 to 995
Data columns (total 8 columns):
user_id     985 non-null object
time        985 non-null object
bet         985 non-null float64
win         985 non-null float64
datetime    985 non-null datetime64[ns]
weekday     985 non-null float64
daytime     985 non-null object
net         985 non-null float64
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 69.3+ KB


In [71]:
df_users.columns = ['raw_user']

In [72]:
df_users.head()

,raw_user
0,User_943\tAccumanst@gmail.com\tИжевск
1,User_908\tAdvismowr@mail.ru\tИжевск
2,User_962\tAnachso@ukr.net\tКраснодар
3,User_973\tAntecia@inbox.ru\tПермь
4,User_902\tBalliaryva@ukr.net\t


In [80]:
for i, field in enumerate(['user_id', 'email', 'city']):
    df_users[field] = df_users.raw_user.apply(lambda x: x.split('\t')[i].lower())

In [81]:
df_users.head()

,raw_user,user_id,email,city
0,User_943\tAccumanst@gmail.com\tИжевск,user_943,accumanst@gmail.com,ижевск
1,User_908\tAdvismowr@mail.ru\tИжевск,user_908,advismowr@mail.ru,ижевск
2,User_962\tAnachso@ukr.net\tКраснодар,user_962,anachso@ukr.net,краснодар
3,User_973\tAntecia@inbox.ru\tПермь,user_973,antecia@inbox.ru,пермь
4,User_902\tBalliaryva@ukr.net\t,user_902,balliaryva@ukr.net,


In [84]:
df_users['user_id'] = df_users.user_id.apply(lambda x: int(x.split('_')[1]))

In [85]:
df_users.head()

,raw_user,user_id,email,city
0,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск
1,User_908\tAdvismowr@mail.ru\tИжевск,908,advismowr@mail.ru,ижевск
2,User_962\tAnachso@ukr.net\tКраснодар,962,anachso@ukr.net,краснодар
3,User_973\tAntecia@inbox.ru\tПермь,973,antecia@inbox.ru,пермь
4,User_902\tBalliaryva@ukr.net\t,902,balliaryva@ukr.net,


In [86]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
raw_user    100 non-null object
user_id     100 non-null int64
email       100 non-null object
city        100 non-null object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [88]:
df_logs['user_id'] = df_logs.user_id.apply(lambda x: int(x.split('_')[1]))

In [89]:
df_logs.head()

,user_id,time,bet,win,datetime,weekday,daytime,net
0,919,2019-01-01 14:06:51,0.0,0.0,2019-01-01 14:06:51,1.0,день,0.0
1,973,2019-01-01 14:51:16,0.0,0.0,2019-01-01 14:51:16,1.0,день,0.0
2,903,2019-01-01 16:31:16,0.0,0.0,2019-01-01 16:31:16,1.0,день,0.0
3,954,2019-01-01 17:17:51,0.0,0.0,2019-01-01 17:17:51,1.0,день,0.0
4,954,2019-01-01 21:31:18,0.0,0.0,2019-01-01 21:31:18,1.0,вечер,0.0


In [90]:
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 0 to 995
Data columns (total 8 columns):
user_id     985 non-null int64
time        985 non-null object
bet         985 non-null float64
win         985 non-null float64
datetime    985 non-null datetime64[ns]
weekday     985 non-null float64
daytime     985 non-null object
net         985 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 69.3+ KB


In [93]:
merged = pd.merge(df_users, df_logs, on='user_id') 

In [94]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 11 columns):
raw_user    975 non-null object
user_id     975 non-null int64
email       975 non-null object
city        975 non-null object
time        975 non-null object
bet         975 non-null float64
win         975 non-null float64
datetime    975 non-null datetime64[ns]
weekday     975 non-null float64
daytime     975 non-null object
net         975 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 91.4+ KB


In [95]:
df_logs.count()

user_id     985
time        985
bet         985
win         985
datetime    985
weekday     985
daytime     985
net         985
dtype: int64

In [97]:
merged.head()

,raw_user,user_id,email,city,time,bet,win,datetime,weekday,daytime,net
0,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-01-06 21:06:57,0.0,0.0,2019-01-06 21:06:57,6.0,вечер,0.0
1,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-01-10 12:16:22,100.0,0.0,2019-01-10 12:16:22,3.0,день,-100.0
2,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-01-27 1:18:47,100.0,0.0,2019-01-27 01:18:47,6.0,ночь,-100.0
3,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-02-10 11:09:24,100.0,5456.0,2019-02-10 11:09:24,6.0,утро,5356.0
4,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-02-12 15:09:38,300.0,5245.0,2019-02-12 15:09:38,1.0,день,4945.0


Задание 1
0/1 point (graded)
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).

Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.

In [114]:
merged.groupby('user_id').net.sum().median()

1986.0

Задание 2*
1 point possible (graded)
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).

Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, что у этого человека все-таки есть хотя бы одна ставка? Например: Человек посетил букмекерскую контору 5 раз из них 1 раз сделал ставку, 4 раза нет - условие выполняется. Человек посетил букмекерскую контору 5 раз из них ни разу ставку не сделал - условие не выполняется. Для того, чтобы узнать это, просуммируйте в каждой группе количество записей со ставкой, равной 0, и поделите на общее количество групп. Если при этом в группе нет записей со ставкой больше 0, считаем количество записей в данной группе равным 0.

In [154]:
math.trunc(merged[merged.user_id.isin(merged[merged.bet > 0].user_id.unique()) & merged.bet == 0].groupby('user_id').count().mean()[0])

5

Задание 3**
1 point possible (graded)
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа - количества дней.

Сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой? Считать нужно только тех, кто делал ставку. Для того, чтобы узнать это, напишите метод, считающий минимальное время среди ставок, равных 0, и минимальное время среди ставок больше 0. После этого верните разницу между вторым и первым числом. Пройдитесь по всем группам в цикле. Если в группе нет ставок больше 0, пропустите эту группу. Просуммируйте разницу во времени для каждой группы (с помощью метода, описанного выше) и поделите на количество групп, которые вы не пропустили.
Например, если ваш результат Timedelta('23 days 12:24:32'), то в поле пишите 23.

In [158]:
only_bet_users = merged[merged.user_id.isin(merged[merged.bet > 0].user_id.unique())]

In [185]:
only_bet_users['max_time_with_zero_bet'] = only_bet_users.apply(lambda x: only_bet_users[(only_bet_users.user_id == x.user_id) & (only_bet_users.bet == 0)].time.max(), axis=1)

In [186]:
only_bet_users.head()

,raw_user,user_id,email,city,time,bet,win,datetime,weekday,daytime,net,first_bet_time,max_time_with_zero_bet
0,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-01-06 21:06:57,0.0,0.0,2019-01-06 21:06:57,6.0,вечер,0.0,2019-01-06 21:06:57,2019-01-06 21:06:57
1,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-01-10 12:16:22,100.0,0.0,2019-01-10 12:16:22,3.0,день,-100.0,2019-01-06 21:06:57,2019-01-06 21:06:57
2,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-01-27 1:18:47,100.0,0.0,2019-01-27 01:18:47,6.0,ночь,-100.0,2019-01-06 21:06:57,2019-01-06 21:06:57
3,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-02-10 11:09:24,100.0,5456.0,2019-02-10 11:09:24,6.0,утро,5356.0,2019-01-06 21:06:57,2019-01-06 21:06:57
4,User_943\tAccumanst@gmail.com\tИжевск,943,accumanst@gmail.com,ижевск,2019-02-12 15:09:38,300.0,5245.0,2019-02-12 15:09:38,1.0,день,4945.0,2019-01-06 21:06:57,2019-01-06 21:06:57


In [191]:
only_bet_users['min_time_with_zero_bet'] = only_bet_users.apply(lambda x: pd.to_datetime(only_bet_users[(only_bet_users.user_id == x.user_id) & (only_bet_users.bet == 0)].time.min()), axis=1)

In [190]:
only_bet_users['min_time_with_nonzero_bet'] = only_bet_users.apply(lambda x: pd.to_datetime(only_bet_users[(only_bet_users.user_id == x.user_id) & (only_bet_users.bet > 0)].time.min()), axis=1)

In [201]:
only_bet_users['first_bet_time'] = only_bet_users.min_time_with_zero_bet - only_bet_users.min_time_with_nonzero_bet

In [202]:
only_bet_users['first_bet_time'].mean()

Timedelta('-46 days +23:37:52.307692')

In [200]:
only_bet_users[only_bet_users.user_id == 944]

,raw_user,user_id,email,city,time,bet,win,datetime,weekday,daytime,net,first_bet_time,max_time_with_zero_bet,min_time_with_zero_bet,min_time_with_nonzero_bet
835,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-01-07 19:45:03,0.0,0.0,2019-01-07 19:45:03,0.0,вечер,0.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
836,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-01-16 19:58:56,0.0,0.0,2019-01-16 19:58:56,2.0,вечер,0.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
837,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-02-15 22:17:01,0.0,0.0,2019-02-15 22:17:01,4.0,вечер,0.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
838,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-02-25 19:53:44,0.0,0.0,2019-02-25 19:53:44,0.0,вечер,0.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
839,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-03-05 21:49:31,100.0,4119.0,2019-03-05 21:49:31,1.0,вечер,4019.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
840,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-03-05 4:29:48,0.0,0.0,2019-03-05 04:29:48,1.0,ночь,0.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
841,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-03-22 19:02:28,500.0,5217.0,2019-03-22 19:02:28,4.0,вечер,4717.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31
842,User_944\tSeenomvura@yahoo.com\tЕкатеринбург,944,seenomvura@yahoo.com,екатеринбург,2019-04-15 9:19:57,700.0,5453.0,2019-04-15 09:19:57,0.0,утро,4753.0,57 days 02:04:28,2019-03-05 4:29:48,2019-01-07 19:45:03,2019-03-05 21:49:31


In [206]:
merged.drop('raw_user', axis=1, inplace=True)
only_bet_users.drop('raw_user', axis=1, inplace=True)

In [207]:
merged.head()

,user_id,email,city,time,bet,win,datetime,weekday,daytime,net
0,943,accumanst@gmail.com,ижевск,2019-01-06 21:06:57,0.0,0.0,2019-01-06 21:06:57,6.0,вечер,0.0
1,943,accumanst@gmail.com,ижевск,2019-01-10 12:16:22,100.0,0.0,2019-01-10 12:16:22,3.0,день,-100.0
2,943,accumanst@gmail.com,ижевск,2019-01-27 1:18:47,100.0,0.0,2019-01-27 01:18:47,6.0,ночь,-100.0
3,943,accumanst@gmail.com,ижевск,2019-02-10 11:09:24,100.0,5456.0,2019-02-10 11:09:24,6.0,утро,5356.0
4,943,accumanst@gmail.com,ижевск,2019-02-12 15:09:38,300.0,5245.0,2019-02-12 15:09:38,1.0,день,4945.0


Задание 1
1 point possible (graded)
Используйте датасет, который получился в результате всех преобразований в прошлой секции (в том числе, заполнение пропусков).

Ответ запишите в поле ниже в виде одного слова, с большой буквы.

Наибольший суммарный выигрыш среди всех городов имеет Москва. Посчитайте следующий за ней город. Для этого сгруппируйте по городам, возьмите признак win, просуммируйте по каждому городу, отсортируйте и получите второй город.

In [213]:
merged.groupby('city').win.sum().sort_values(ascending=False)

city
москва             11959741.0
воронеж              184338.0
санкт-петербург      151007.0
казань                97806.0
ярославль             97441.0
ижевск                84895.0
красноярск            84767.0
арзангелтск           74375.0
пермь                 67734.0
хабаровск             65459.0
краснодар             62718.0
ставрополь            46003.0
                      45361.0
екатеринбург          36682.0
тюмень                 4701.0
Name: win, dtype: float64

Задание 2*
1 point possible (graded)
Используйте датасет, который получился в результате всех преобразований в прошлой секции (в том числе, заполнение пропусков).

Ответ запишите в поле ниже в виде целого числа (нужно отбросить дробную часть).

Подсказка: можно использовать методы min() и max().

Во сколько раз различаются в среднем максимальная и минимальная ставки по городам? Для того, чтобы это посчитать, нужно сгруппировать по городам, взять среднее от признака bet, найти максимальное и минимальное значения, затем поделить одно на другое.

In [223]:
tmp_df = merged.groupby('city').bet.mean()
math.trunc(max(tmp_df)/min(tmp_df))

227